In [32]:
import os
import io
import cv2
import pandas as pd
import shutil

In [3]:
# 📂 Crear carpetas para imágenes y anotaciones
os.makedirs("yolo_dataset/images/train", exist_ok=True)
os.makedirs("yolo_dataset/labels/train", exist_ok=True)
os.makedirs("yolo_dataset/images/val", exist_ok=True)
os.makedirs("yolo_dataset/labels/val", exist_ok=True)

In [4]:
# Leer csv
df_train = pd.read_csv('train.csv')
df_test = pd.read_csv('test.csv')

In [8]:
df_train

,filename,width,height,class,xmin,ymin,xmax,ymax
0,0a836954ceeac15f_jpg.rf.7f4bcc9a1e1d2db7dabcd6...,416,416,carro,303.000000,220.500000,416.000000,386.500000
1,0a836954ceeac15f_jpg.rf.7f4bcc9a1e1d2db7dabcd6...,416,416,carro,6.000000,235.500000,250.000000,351.500000
2,1f133e6ee29e99d0_jpg.rf.c3edd22ab67ab12d42c66f...,416,416,carro,122.000000,3.500000,346.000000,299.500000
3,0c50d905a9db3118_jpg.rf.G8X5TeUEj7ElWy0jRCa4.jpg,416,416,carro,71.000000,195.500000,166.000000,295.500000
4,0c50d905a9db3118_jpg.rf.G8X5TeUEj7ElWy0jRCa4.jpg,416,416,carro,19.000000,205.500000,49.000000,238.500000
...,...,...,...,...,...,...,...,...
693,motocarro1_1.jpg,1469,921,moto,867.167726,461.085133,1091.858958,753.651842
694,motocarro1_1.jpg,1469,921,moto,1066.113088,605.027954,1307.188056,908.127065
695,motocarro1_1.jpg,1469,921,moto,1226.439644,359.271919,1431.236341,665.881830
696,motocarro1_1.jpg,1469,921,moto,1184.310038,100.642948,1390.277001,322.993647


In [33]:
def Agrupacion (df,path,dataset):
# 🔁 Agrupar anotaciones por imagen
  for img_file, group in df.groupby("filename"):
      # Leer imagen
      image_path = f"{path}\\{img_file}"
      if not os.path.exists(image_path):
          print(f"Imagen no encontrada: {image_path}")
          continue

      img = cv2.imread(image_path)
      h, w = img.shape[:2]

    
      # Construye la ruta destino correctamente
      dest_path = os.path.join("yolo_dataset", "images", dataset, img_file)

      # Copia la imagen
      shutil.copy(image_path, dest_path)

      # Crear archivo de anotaciones
      label_path = f"yolo_dataset\\labels\\{dataset}\\{os.path.splitext(img_file)[0]}.txt"
      with open(label_path, "w") as f:
          for _, row in group.iterrows():
              class_id = row['class']
              x_center = ((row['xmin'] + row['xmax']) / 2) / w
              y_center = ((row['ymin'] + row['ymax']) / 2) / h
              box_width = (row['xmax'] - row['xmin']) / w
              box_height = (row['ymax'] - row['ymin']) / h
              f.write(f"{class_id} {x_center:.6f} {y_center:.6f} {box_width:.6f} {box_height:.6f}\n")


In [6]:
# 🔧 Crear un diccionario de clases
class_names = df_train['class'].unique().tolist()
class_dict = {name: i for i, name in enumerate(class_names)}

In [38]:
# direccion de las imagenes train y test

path_train = 'Finaldatset\\Finaldatset\\train'
path_test = 'Finaldatset\\Finaldatset\\test'
dataset = 'val'

In [39]:
Agrupacion(df_test,path_test,dataset)

In [40]:
import yaml

with open("yolo_dataset/data.yaml", "w") as f:
    yaml.dump({
        'path': os.path.abspath("yolo_dataset"),
        'train': 'images/train',
        'val': 'images/val',  # cambia si tienes otro set de validación
        'nc': len(class_names),
        'names': class_names
    }, f)